## Import

In [1]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np
from numpy import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import DualTransform,ImageOnlyTransform

import random
import time
import warnings
import sys
import argparse
import shutil

import torch.backends.cudnn as cudnn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import LambdaLR

import torch.nn as nn
from torch.hub import load_state_dict_from_url
from torchvision import models
from collections import OrderedDict
from pytorch_model_summary import summary
from randaugment import RandAugment
from scipy import stats
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torch.nn.utils.spectral_norm import spectral_norm

import io
import glob
import time
import zipfile
import subprocess
import logging
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from contextlib import contextmanager
import torch
import torch.distributed as dist
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
#config
#train_source, val_source: (2048,1024)
#train_target,test : (1920,1080)

#class 정보
# 0 : Road
# 1 : Sidewalk
# 2 : Construction
# 3 : Fence
# 4 : Pole 
# 5 : Traffic Light
# 6 : Traffic Sign
# 7 : Nature
# 8 : Sky
# 9 : Person
# 10 : Rider
# 11 : Car

batch_size = 2
f_batch_size = 2
epochs = 50
F_epochs = 50
num_class = 13
source_root = './home/js/ssai/open/data/train_source_image'
target_root = './home/js/ssai/open/data/train_target_image'
learning_rate = 2.5e-2 #2.5e-3
iters_per_epoch = 1000
seed = 42
weight_decay = 0.0005
momentum = 0.9
train_size = (512,512)
test_output_size = (960,540)
lr_power = 0.9
lr_d =1e-4
ignore_label = 255
trade_off = 0.01 #0.001 
pyramid_weight = 0.5
l1_weight = 6

palette = [[0, 94, 135], [242, 255, 97], [165, 42, 42], [0, 0, 192],
                [197, 226, 255], [0, 60, 100], [0, 0, 142], [62, 200, 71],
                [255,207,157], [0, 187, 255], [255, 102, 163],[166,97,247],[0,0,0]]

palette = np.array(palette)

## Utils

In [3]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Define Model

In [4]:
#deeplabv2

model_urls = {
    'deeplabv2_resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth'
}

#https://discuss.pytorch.org/t/how-to-load-load-state-dict-for-resnet-model/125998/2
affine_par = True

class ResNet(nn.Module):
    def __init__(self, block, layers):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)  # change

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1, dilation=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=4)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion or dilation == 2 or dilation == 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, affine=affine_par))
        for i in downsample._modules['1'].parameters():
            i.requires_grad = False
        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False)  # change
        self.bn1 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn1.parameters():
            i.requires_grad = False

        padding = dilation

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,  # change
                               padding=padding, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes, affine=affine_par)
        for i in self.bn2.parameters():
            i.requires_grad = False

        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4, affine=affine_par)
        for i in self.bn3.parameters():
            i.requires_grad = False

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ASPP_V2(nn.Module):
    def __init__(self, inplanes, dilation_series, padding_series, num_classes):
        super(ASPP_V2, self).__init__()
        self.conv2d_list = nn.ModuleList()
        for dilation, padding in zip(dilation_series, padding_series):
            self.conv2d_list.append(
                nn.Conv2d(inplanes, num_classes, kernel_size=3, stride=1, padding=padding, dilation=dilation,
                          bias=True))
        for m in self.conv2d_list:
            m.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.conv2d_list[0](x)
        for i in range(len(self.conv2d_list) - 1):
            out += self.conv2d_list[i + 1](x)

        return out


class Deeplab(nn.Module):
    def __init__(self, backbone, classifier, num_classes):
        super(Deeplab, self).__init__()
        self.backbone = backbone
        self.classifier = classifier
        self.num_classes = num_classes
        self.conv1 = nn.Conv2d(in_channels=5, out_channels=3, kernel_size=1, bias=False)

    def forward(self, x):
        if  x.size()[1] != 3 :
             x = self.conv1(x)
        x = self.backbone(x)
        y = self.classifier(x)
        return y

    def get_1x_lr_params_NOscale(self):
        """
        This generator returns all the parameters of the net except for
        the last classification layer. Note that for each batchnorm layer,
        requires_grad is set to False in deeplab_resnet.py, therefore this function does not return
        any batchnorm parameter
        """
        # layers = [self.backbone.conv1, self.backbone.bn1,
        #         self.backbone.layer1, self.backbone.layer2, self.backbone.layer3, self.backbone.layer4]
        layers = [list(self.backbone.children())[0], list(self.backbone.children())[1],
                list(self.backbone.children())[4], list(self.backbone.children())[5], 
                list(self.backbone.children())[6], list(self.backbone.children())[7]]
        for layer in layers:
            for module in layer.modules():
                for param in module.parameters():
                    if param.requires_grad:
                        yield param

    def get_10x_lr_params(self):
        """
        This generator returns all the parameters for the last layer of the net,
        which does the classification of pixel into classes
        """
        for param in self.classifier.parameters():
            yield param

    def get_parameters(self, lr=1.):
        return [
            {'params': self.get_1x_lr_params_NOscale(), 'lr': 0.1 * lr},
            {'params': self.get_10x_lr_params(), 'lr': lr}
        ]

def deeplabv2_resnet101(num_classes=13, pretrained_backbone=True):
    """Constructs a DeepLabV2 model with a ResNet-1 01 backbone.

     Args:
         num_classes (int, optional): number of classes. Default: 19
         pretrained_backbone (bool, optional): If True, returns a model pre-trained on ImageNet. Default: True.
     """
    backbone = ResNet(Bottleneck, [3, 4, 23, 3])
    if pretrained_backbone:
        # download from Internet
        #saved_state_dict = load_state_dict_from_url(model_urls['deeplabv2_resnet101'], map_location=lambda storage, loc: storage, file_name="deeplabv2_resnet101.pth")
        #pre_backbone= models.resnet101(pretrained=True)
        #pre_backbone = nn.Sequential(*list(backbone.children())[:-2])
        ckpt = torch.load('./cityscapes_epoch_59.pth')
        
        # state_dict = ckpt['model']
        new_state_dict = OrderedDict()
        for k, v in ckpt.items():
            if not k.split('.')[1] == 'layer5':
                name =  k.replace("Scale.","")
                new_state_dict[name] = v
            
        backbone.load_state_dict(new_state_dict)
        #new_params = backbone.state_dict().copy()
        # for i in saved_state_dict:
        #     i_parts = i.split('.')
        #     if not i_parts[1] == 'layer5':
        #         new_params['.'.join(i_parts[1:])] = saved_state_dict[i]
        # backbone.load_state_dict(new_params)

    classifier = ASPP_V2(2048, [6, 12, 18, 24], [6, 12, 18, 24], num_classes)
    return Deeplab(backbone, classifier, num_classes)


In [5]:
from typing import Optional, List


class AverageMeter(object):
    r"""Computes and stores the average and current value.

    Examples::

        >>> # Initialize a meter to record loss
        >>> losses = AverageMeter()
        >>> # Update meter after every minibatch update
        >>> losses.update(loss_value, batch_size)
    """
    def __init__(self, name: str, fmt: Optional[str] = ':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.count > 0:
            self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class AverageMeterDict(object):
    def __init__(self, names: List, fmt: Optional[str] = ':f'):
        self.dict = {
            name: AverageMeter(name, fmt) for name in names
        }

    def reset(self):
        for meter in self.dict.values():
            meter.reset()

    def update(self, accuracies, n=1):
        for name, acc in accuracies.items():
            self.dict[name].update(acc, n)

    def average(self):
        return {
            name: meter.avg for name, meter in self.dict.items()
        }

    def __getitem__(self, item):
        return self.dict[item]


class Meter(object):
    """Computes and stores the current value."""
    def __init__(self, name: str, fmt: Optional[str] = ':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0

    def update(self, val):
        self.val = val

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        #print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [6]:
import torch
import prettytable

__all__ = ['keypoint_detection']

def binary_accuracy(output: torch.Tensor, target: torch.Tensor) -> float:
    """Computes the accuracy for binary classification"""
    with torch.no_grad():
        batch_size = target.size(0)
        pred = (output >= 0.5).float().t().view(-1)
        correct = pred.eq(target.view(-1)).float().sum()
        correct.mul_(100. / batch_size)
        return correct


def accuracy(output, target, topk=(1,)):
    r"""
    Computes the accuracy over the k top predictions for the specified values of k

    Args:
        output (tensor): Classification outputs, :math:`(N, C)` where `C = number of classes`
        target (tensor): :math:`(N)` where each value is :math:`0 \leq \text{targets}[i] \leq C-1`
        topk (sequence[int]): A list of top-N number.

    Returns:
        Top-N accuracies (N :math:`\in` topK).
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target[None])

        res = []
        for k in topk:
            correct_k = correct[:k].flatten().sum(dtype=torch.float32)
            res.append(correct_k * (100.0 / batch_size))
        return res


class ConfusionMatrix(object):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.mat = None

    def update(self, target, output):
        """
        Update confusion matrix.

        Args:
            target: ground truth
            output: predictions of models

        Shape:
            - target: :math:`(minibatch, C)` where C means the number of classes.
            - output: :math:`(minibatch, C)` where C means the number of classes.
        """
        n = self.num_classes
        if self.mat is None:
            self.mat = torch.zeros((n, n), dtype=torch.int64, device=target.device)
        with torch.no_grad():
            k = (target >= 0) & (target < n)
            inds = n * target[k].to(torch.int64) + output[k]
            self.mat += torch.bincount(inds, minlength=n**2).reshape(n, n)

    def reset(self):
        self.mat.zero_()

    def compute(self):
        """compute global accuracy, per-class accuracy and per-class IoU"""
        h = self.mat.float()
        acc_global = torch.diag(h).sum() / h.sum()
        acc = torch.diag(h) / h.sum(1)
        iu = torch.diag(h) / (h.sum(1) + h.sum(0) - torch.diag(h))
        return acc_global, acc, iu

    # def reduce_from_all_processes(self):
    #     if not torch.distributed.is_available():
    #         return
    #     if not torch.distributed.is_initialized():
    #         return
    #     torch.distributed.barrier()
    #     torch.distributed.all_reduce(self.mat)

    def __str__(self):
        acc_global, acc, iu = self.compute()
        return (
            'global correct: {:.1f}\n'
            'average row correct: {}\n'
            'IoU: {}\n'
            'mean IoU: {:.1f}').format(
                acc_global.item() * 100,
                ['{:.1f}'.format(i) for i in (acc * 100).tolist()],
                ['{:.1f}'.format(i) for i in (iu * 100).tolist()],
                iu.mean().item() * 100)

    def format(self, classes: list):
        """Get the accuracy and IoU for each class in the table format"""
        acc_global, acc, iu = self.compute()

        table = prettytable.PrettyTable(["class", "acc", "iou"])
        for i, class_name, per_acc, per_iu in zip(range(len(classes)), classes, (acc * 100).tolist(), (iu * 100).tolist()):
            table.add_row([class_name, per_acc, per_iu])

        return 'global correct: {:.1f}\nmean correct:{:.1f}\nmean IoU: {:.1f}\n{}'.format(
            acc_global.item() * 100, acc.mean().item() * 100, iu.mean().item() * 100, table.get_string())



In [7]:
def send_to_device(tensor, device):
    """
    Recursively sends the elements in a nested list/tuple/dictionary of tensors to a given device.

    Args:
        tensor (nested list/tuple/dictionary of :obj:`torch.Tensor`):
            The data to send to a given device.
        device (:obj:`torch.device`):
            The device to send the data to

    Returns:
        The same data structure as :obj:`tensor` with all tensors sent to the proper device.
    """
    if isinstance(tensor, (list, tuple)):
        return type(tensor)(send_to_device(t, device) for t in tensor)
    elif isinstance(tensor, dict):
        return type(tensor)({k: send_to_device(v, device) for k, v in tensor.items()})
    elif not hasattr(tensor, "to"):
        return tensor
    return tensor.to(device)


class ForeverDataIterator:
    r"""A data iterator that will never stop producing data"""

    def __init__(self, data_loader: DataLoader, device=None):
        self.data_loader = data_loader
        self.iter = iter(self.data_loader)
        self.device = device

    def __next__(self):
        try:
            data = next(self.iter)
            if self.device is not None:
                data = send_to_device(data, self.device)
        except StopIteration:
            self.iter = iter(self.data_loader)
            data = next(self.iter)
            if self.device is not None:
                data = send_to_device(data, self.device)
        return data

    def __len__(self):
        return len(self.data_loader)

## Model Train

In [8]:
class MaxSquareloss(nn.Module):
    def __init__(self, ignore_index= -1, num_class=13):
        super().__init__()
        self.ignore_index = ignore_index
        self.num_class = num_class
    
    def forward(self, pred, prob):
        """
        :param pred: predictions (N, C, H, W)
        :param prob: probability of pred (N, C, H, W)
        :return: maximum squares loss
        """
        # prob -= 0.5
        mask = (prob != self.ignore_index)    
        loss = -torch.mean(torch.pow(prob, 2)[mask]) / 2
        return loss

In [9]:
#train

def train(train_source_iter: ForeverDataIterator, train_target_iter: ForeverDataIterator,
          model, interp, criterion, dann, flownet,
          optimizer: SGD, lr_scheduler: LambdaLR, optimizer_d: SGD, lr_scheduler_d: LambdaLR,
          epoch: int):
    
    losses_s = AverageMeter('Loss (s)', ':3.2f')
    losses_transfer = AverageMeter('Loss (transfer)', ':3.2f')
    losses_discriminator = AverageMeter('Loss (discriminator)', ':3.2f')
    accuracies_s = Meter('Acc (s)', ':3.2f')
    accuracies_t = Meter('Acc (t)', ':3.2f')
    iou_s = Meter('IoU (s)', ':3.2f')
    iou_t = Meter('IoU (t)', ':3.2f')

    confmat_s = ConfusionMatrix(num_class)
    #confmat_t = ConfusionMatrix(model.num_classes)

    model.train()
    flownet.eval()
    for i in range(iters_per_epoch):
        if(i % 100 == 0):
            print("iters : ",i)
        iter_start = time.time()
        x_s, label_s = next(train_source_iter)
        x_t = next(train_target_iter)

        x_s = x_s.to(device)
        label_s = label_s.long().to(device)

        x_t = x_t.to(device)
        #label_t = label_t.long().to(device)
        
        _,_,target_flow_map = flownet(x_t)
        
        # print(x_t.shape)
        # print(target_flow_map.shape)
        
        x_t_flow = torch.concat((x_t,target_flow_map),dim=1)
        # print(x_t_flow.shape)
        optimizer.zero_grad()
        optimizer_d.zero_grad()

        # # Step 1: Train the segmentation network, freeze the discriminator
        dann.eval() #DomainAdversarialEntropyLoss
        y_s = model(x_s)
        pred_s = interp(y_s)
        #segmentaiton : cross-entropy loss (generator 즉 segmentataion network 학습)
        loss_cls_s = criterion(pred_s, label_s.squeeze(1))
        loss_cls_s.backward()

        # adversarial training to fool the discriminator

        y_t = model(x_t_flow) # 똑같이 segmentation network에 target image를 넣고
        pred_t = interp(y_t)
        loss_transfer = dann(pred_t, 'source') # target image의 prediction 값을
        (loss_transfer * trade_off).backward()

        # Step 2: Train the discriminator
        dann.train()
        loss_discriminator = 0.5 * (dann(pred_s.detach(), 'source') + dann(pred_t.detach(), 'target'))
        loss_discriminator.backward()

        # compute gradient and do SGD step
        optimizer.step()
        optimizer_d.step()
        lr_scheduler.step()
        lr_scheduler_d.step()

        # measure accuracy and record loss
        losses_s.update(loss_cls_s.item(), x_s.size(0))
        losses_transfer.update(loss_transfer.item(), x_s.size(0))
        losses_discriminator.update(loss_discriminator.item(), x_s.size(0))

        confmat_s.update(label_s.flatten(), pred_s.argmax(1).flatten())
        #confmat_t.update(label_t.flatten(), pred_t.argmax(1).flatten())
        acc_global_s, acc_s, iu_s = confmat_s.compute()
        #acc_global_t, acc_t, iu_t = confmat_t.compute()
        accuracies_s.update(acc_s.mean().item())
        #accuracies_t.update(acc_t.mean().item())
        iou_s.update(iu_s.mean().item())

        iter_end = time.time()
        if(i % 100 == 0):
            print("seg_loss : ", loss_cls_s.item())
            print("entropy_loss: ",loss_transfer.item())
            print("disc_loss : ", loss_discriminator.item())
            print("iter end, time : ", iter_end-iter_start)
        #iou_t.update(iu_t.mean().item())

        # if i % args.print_freq == 0:
        #     progress.display(i)

        #     if visualize is not None:
        #         visualize(x_s[0], pred_s[0], label_s[0], "source_{}".format(i))
        #         visualize(x_t[0], pred_t[0], label_t[0], "target_{}".format(i))

def validate(val_loader: DataLoader, model, interp, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    acc = Meter('Acc', ':3.2f')
    iou = Meter('IoU', ':3.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, acc, iou],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()
    confmat = ConfusionMatrix(num_class)

    with torch.no_grad():
        for i, (x, label) in enumerate(val_loader):
            x = x.to(device)
            label = label.long().to(device)

            # compute output
            output = interp(model(x))
            loss = criterion(output, label)

            # measure accuracy and record loss
            losses.update(loss.item(), x.size(0))
            confmat.update(label.flatten(), output.argmax(1).flatten())
            acc_global, accs, iu = confmat.compute()
            acc.update(accs.mean().item())
            iou.update(iu.mean().item())

            # if i % args.print_freq == 0:
            #     progress.display(i)

            #     if visualize is not None:
            #         visualize(x[0], output[0], label[0], "val_{}".format(i))

    return confmat

In [10]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False, source=True):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.source = source
        self.classes = ['Road','Sidewalk','Construction','Fence','Pole','Traffic Light'
                        ,'Traffic Sign','Nature','Sky','Person','Rider','Car','Background']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        if self.source==True:
            mask_path = self.data.iloc[idx, 2]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

            if self.transform:
            
                augmented = self.transform(image=image, mask=mask)
                image = augmented['image']
                mask = augmented['mask']

                #transform 확인용
                
                # mask = palette[mask]
                # name = img_path.split('/')[-1]
                # cv2.imwrite(f'./trans/{name}.png', image)
                # cv2.imwrite(f'./trans/mask_{name}.png', mask)
            return image, mask
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image

        
    

In [11]:
# fisheye augmentation 

scale = 5.5
def fisheye_distortion_rm(image):
    
    h,w = image.shape[:2]

    focal_length = w / 4
    center_x = w / 2 
    center_y = h / 2
    camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

    dist_coeffs = np.array([0,0.5,0,0],dtype=np.float32)
    map_x, map_y = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, None, (w, h), cv2.CV_32FC1)
    undistorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    uh,uw = undistorted_image.shape[:2]

    # min_x = np.where((undistorted_image[uh//2][:][:]!=0))[0][0]
    # max_x = np.where((undistorted_image[uh//2][:][:]!=0))[0][-1]
    # min_y = np.where((undistorted_image[:,uw//2,:]!=0))[0][0]
    # max_y = np.where((undistorted_image[:,uw//2,:]!=0))[0][-1]

    # undistorted_image = undistorted_image[min_y:max_y,min_x:max_x]

    undistorted_image = undistorted_image[int(h/scale):int(h-h/scale),int(w/scale):int(w-w/scale)]

    return undistorted_image

def fisheye_distortion_mask_rm(image):
    #image[image==0] = 255
    h,w = image.shape[:2]

    focal_length = w / 4
    center_x = w / 2 
    center_y = h / 2
    camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

    dist_coeffs = np.array([0,0.5,0,0],dtype=np.float32)

    map_x, map_y = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, None, (w, h), cv2.CV_32FC1)
    undistorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT,borderValue=12)


    uh,uw = undistorted_image.shape[:2]

    # min_x = np.where((undistorted_image[uh//2][:][:]!=0))[0][0]
    # max_x = np.where((undistorted_image[uh//2][:][:]!=0))[0][-1]
    # min_y = np.where((undistorted_image[:,uw//2,:]!=0))[0][0]
    # max_y = np.where((undistorted_image[:,uw//2,:]!=0))[0][-1]

    undistorted_image = undistorted_image[int(h/scale):int(h-h/scale),int(w/scale):int(w-w/scale)].astype(np.uint8)
    #undistorted_image = undistorted_image[coord[0]:coord[1],coord[2]:coord[3]].astype(np.uint8)

    return undistorted_image

class Fisheye(DualTransform):
    def __init__(self):
        super(Fisheye,self).__init__()

    def apply(self, img, **params):
        return fisheye_distortion_rm(img)

    def apply_to_mask(self, mask, **params):
        return fisheye_distortion_mask_rm(mask)
    
# def fisheye_distortion(image):
    
#     h,w = image.shape[:2]

#     focal_length = w / 4
#     center_x = w / 2 
#     center_y = h / 2
#     camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

#     dist_coeffs = np.array([0,0.5,0,0],dtype=np.uint8)

#     undistorted_image = cv2.undistort(image,camera_matrix,dist_coeffs)
#     #undistorted_image = undistorted_image[h//5:h-h//5,w//5:w-w//5]

#     return undistorted_image

# def fisheye_distortion_mask(image):
#     image[image==0] = 255
#     h,w = image.shape[:2]

#     focal_length = w / 4
#     center_x = w / 2 
#     center_y = h / 2
#     camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

#     dist_coeffs = np.array([0,0.5,0,0],dtype=np.float32)

#     undistorted_image = cv2.undistort(image,camera_matrix,dist_coeffs)
#     #undistorted_image = undistorted_image[h//5:h-h//5,w//5:w-w//5].astype(np.uint8)
#     #undistorted_image = np.round(undistorted_image).astype(np.uint8)
#     undistorted_image[undistorted_image==0] = 12
#     #undistorted_image = filter(undistorted_image)
#     undistorted_image[undistorted_image>12] = 0

#     return undistorted_image
   


In [12]:
def prob_2_entropy(prob):
    """ convert probabilistic prediction maps to weighted self-information maps
    """
    n, c, h, w = prob.size()
    return -torch.mul(prob, torch.log2(prob + 1e-30)) / np.log2(c)


def bce_loss(y_pred, y_label):
    y_truth_tensor = torch.FloatTensor(y_pred.size())
    y_truth_tensor.fill_(y_label)
    y_truth_tensor = y_truth_tensor.to(y_pred.get_device())
    return F.binary_cross_entropy_with_logits(y_pred, y_truth_tensor)

class Discriminator(nn.Sequential):
    """
    Domain discriminator model from
    `ADVENT: Adversarial Entropy Minimization for Domain Adaptation in Semantic Segmentation (CVPR 2019) <https://arxiv.org/abs/1811.12833>`_

    Distinguish pixel-by-pixel whether the input predictions come from the source domain or the target domain.
    The source domain label is 1 and the target domain label is 0.

    Args:
        num_classes (int): num of classes in the predictions
        ndf (int): dimension of the hidden features

    Shape:
        - Inputs: :math:`(minibatch, C, H, W)` where :math:`C` is the number of classes
        - Outputs: :math:`(minibatch, 1, H, W)`
    """
    def __init__(self, num_classes, ndf=64):
        super(Discriminator, self).__init__(
            nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, kernel_size=4, stride=2, padding=1),
        )

class DomainAdversarialEntropyLoss(nn.Module):
    r"""The `Domain Adversarial Entropy Loss <https://arxiv.org/abs/1811.12833>`_

    Minimizing entropy with adversarial learning through training a domain discriminator.

    Args:
        domain_discriminator (torch.nn.Module): A domain discriminator object, which predicts
          the domains of predictions. Its input shape is :math:`(minibatch, C, H, W)` and output shape is :math:`(minibatch, 1, H, W)`

    Inputs:
        - logits (tensor): logits output of segmentation model
        - domain_label (str, optional): whether the data comes from source or target.
          Choices: ['source', 'target']. Default: 'source'

    Shape:
        - logits: :math:`(minibatch, C, H, W)` where :math:`C` means the number of classes
        - Outputs: scalar.

    Examples::

        >>> B, C, H, W = 2, 19, 512, 512
        >>> discriminator = Discriminator(num_classes=C)
        >>> dann = DomainAdversarialEntropyLoss(discriminator)
        >>> # logits output on source domain and target domain
        >>> y_s, y_t = torch.randn(B, C, H, W), torch.randn(B, C, H, W)
        >>> loss = 0.5 * (dann(y_s, "source") + dann(y_t, "target"))
    """
    def __init__(self, discriminator: nn.Module):
        super(DomainAdversarialEntropyLoss, self).__init__()
        self.discriminator = discriminator
        self.entropy = None

    def forward(self, logits, domain_label='source'):
        """
        """
        assert domain_label in ['source', 'target']
        probability = F.softmax(logits, dim=1) 
        entropy = prob_2_entropy(probability)
        self.entropy = entropy
        domain_prediciton = self.discriminator(entropy)
        if domain_label == 'source':
            return bce_loss(domain_prediciton, 1)
        else:
            return bce_loss(domain_prediciton, 0)

    def train(self, mode=True):
        r"""Sets the discriminator in training mode. In the training mode,
        all the parameters in discriminator will be set requires_grad=True.

        Args:
            mode (bool): whether to set training mode (``True``) or evaluation mode (``False``). Default: ``True``.
        """
        self.discriminator.train(mode)
        for param in self.discriminator.parameters():
            param.requires_grad = mode
        return self

    def eval(self):
        r"""Sets the module in evaluation mode. In the training mode,
        all the parameters in discriminator will be set requires_grad=False.

        This is equivalent with :meth:`self.train(False) <torch.nn.Module.train>`.
        """
        return self.train(False)

## FlowNet

In [13]:
def use_spectral_norm(module, use_sn=False):
    if use_sn:
        return spectral_norm(module)
    return module

In [14]:
# set parameter to gpu or cpu
def set_device(args):
  if torch.cuda.is_available():
    if isinstance(args, list):
      return (item.cuda() for item in args)
    else:
      return args.cuda()
  return args

def postprocess(img):
  img = (img+1)/2*255
  img = img.permute(0,2,3,1)
  img = img.int().cpu().numpy().astype(np.uint8)
  return img


class Progbar(object):
  """Displays a progress bar.

  Arguments:
    target: Total number of steps expected, None if unknown.
    width: Progress bar width on screen.
    verbose: Verbosity mode, 0 (silent), 1 (verbose), 2 (semi-verbose)
    stateful_metrics: Iterable of string names of metrics that
      should *not* be averaged over time. Metrics in this list
      will be displayed as-is. All others will be averaged
      by the progbar before display.
    interval: Minimum visual progress update interval (in seconds).
  """

  def __init__(self, target, width=25, verbose=1, interval=0.05, stateful_metrics=None):
    self.target = target
    self.width = width
    self.verbose = verbose
    self.interval = interval
    if stateful_metrics:
      self.stateful_metrics = set(stateful_metrics)
    else:
      self.stateful_metrics = set()

    self._dynamic_display = ((hasattr(sys.stdout, 'isatty') and
      sys.stdout.isatty()) or 'ipykernel' in sys.modules or 'posix' in sys.modules)
    self._total_width = 0
    self._seen_so_far = 0
    # We use a dict + list to avoid garbage collection
    # issues found in OrderedDict
    self._values = {}
    self._values_order = []
    self._start = time.time()
    self._last_update = 0

  def update(self, current, values=None):
    """Updates the progress bar.
    Arguments:
      current: Index of current step.
      values: List of tuples:
        `(name, value_for_last_step)`.
        If `name` is in `stateful_metrics`,
        `value_for_last_step` will be displayed as-is.
        Else, an average of the metric over time will be displayed.
    """
    values = values or []
    for k, v in values:
      if k not in self._values_order:
        self._values_order.append(k)
      if k not in self.stateful_metrics:
        if k not in self._values:
          self._values[k] = [v * (current - self._seen_so_far), current - self._seen_so_far]
        else:
          self._values[k][0] += v * (current - self._seen_so_far)
          self._values[k][1] += (current - self._seen_so_far)
      else:
        self._values[k] = v
    self._seen_so_far = current

    now = time.time()
    info = ' - %.0fs' % (now - self._start)
    if self.verbose == 1:
      if (now - self._last_update < self.interval and 
        self.target is not None and current < self.target):
          return

      prev_total_width = self._total_width
      if self._dynamic_display:
        sys.stdout.write('\b' * prev_total_width)
        sys.stdout.write('\r')
      else:
        sys.stdout.write('\n')

      if self.target is not None:
        numdigits = int(np.floor(np.log10(self.target))) + 1
        barstr = '%%%dd/%d [' % (numdigits, self.target)
        bar = barstr % current
        prog = float(current) / self.target
        prog_width = int(self.width * prog)
        if prog_width > 0:
          bar += ('=' * (prog_width - 1))
          if current < self.target:
            bar += '>'
          else:
            bar += '='
        bar += ('.' * (self.width - prog_width))
        bar += ']'
      else:
        bar = '%7d/Unknown' % current
      self._total_width = len(bar)
      sys.stdout.write(bar)
      if current:
        time_per_unit = (now - self._start) / current
      else:
        time_per_unit = 0
      if self.target is not None and current < self.target:
        eta = time_per_unit * (self.target - current)
        if eta > 3600:
          eta_format = '%d:%02d:%02d' % (eta // 3600, (eta % 3600) // 60, eta % 60)
        elif eta > 60:
          eta_format = '%d:%02d' % (eta // 60, eta % 60)
        else:
          eta_format = '%ds' % eta
        info = ' - ETA: %s' % eta_format
      else:
        if time_per_unit >= 1:
          info += ' %.0fs/step' % time_per_unit
        elif time_per_unit >= 1e-3:
          info += ' %.0fms/step' % (time_per_unit * 1e3)
        else:
          info += ' %.0fus/step' % (time_per_unit * 1e6)

      for k in self._values_order:
        info += ' - %s:' % k
        if isinstance(self._values[k], list):
          avg = np.mean(self._values[k][0] / max(1, self._values[k][1]))
          if abs(avg) > 1e-3:
            info += ' %.4f' % avg
          else:
            info += ' %.4e' % avg
        else:
          info += ' %s' % self._values[k]

      self._total_width += len(info)
      if prev_total_width > self._total_width:
        info += (' ' * (prev_total_width - self._total_width))
      if self.target is not None and current >= self.target:
        info += '\n'
      sys.stdout.write(info)
      sys.stdout.flush()
    elif self.verbose == 2:
      if self.target is None or current >= self.target:
        for k in self._values_order:
          info += ' - %s:' % k
          avg = np.mean(self._values[k][0] / max(1, self._values[k][1]))
          if avg > 1e-3:
            info += ' %.4f' % avg
          else:
            info += ' %.4e' % avg
        info += '\n'
        sys.stdout.write(info)
        sys.stdout.flush()
    self._last_update = now

  def add(self, n, values=None):
    self.update(self._seen_so_far + n, values)


In [85]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False, source=True):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.source = source
        self.classes = ['Road','Sidewalk','Construction','Fence','Pole','Traffic Light'
                        ,'Traffic Sign','Nature','Sky','Person','Rider','Car','Background']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = image
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
            #transform 확인용
            # name = img_path.split('/')[-1]
            # cv2.imwrite(f'./trans/{name}.png', image)
            # cv2.imwrite(f'./trans/mask_{name}.png', mask)
            
        return image, mask

In [86]:
def random_fisheye_distortion_rm(image):

    h,w = image.shape[:2]

    fl_rand = random.randint(2,6)
    focal_length = w / fl_rand

    center_x = w / 2 
    center_y = h / 2
    camera_matrix = np.array([[focal_length,0,center_x],[0,focal_length,center_y],[0,0,1]],dtype=np.float32)

    rand_dist = np.random.rand(1)[0]
    dist_coeffs = np.array([0,rand_dist,0,0],dtype=np.float32)

    map_x, map_y = cv2.initUndistortRectifyMap(camera_matrix, dist_coeffs, None, None, (w, h), cv2.CV_32FC1)
    undistorted_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    uh,uw = undistorted_image.shape[:2]


    min_x = np.where((undistorted_image[uh//2][:][:]!=0))[0][0]
    max_x = np.where((undistorted_image[uh//2][:][:]!=0))[0][-1]
    min_y = np.where((undistorted_image[:,uw//2,:]!=0))[0][0]
    max_y = np.where((undistorted_image[:,uw//2,:]!=0))[0][-1]

    undistorted_image = undistorted_image[min_y:max_y,min_x:max_x]
    
    return undistorted_image

def random_fisheye_distortion_mask_rm(image):
    return image

class RandomFisheye(DualTransform):
    def __init__(self):
        super(RandomFisheye,self).__init__()
    
    def apply(self, img, **params):
        return random_fisheye_distortion_rm(img)
    
    def apply_to_mask(self, mask, **params):
        return random_fisheye_distortion_mask_rm(mask)

In [14]:
def get_coordinate(shape, det_uv):
    b, _, w, h = shape
    uv_d = np.zeros([w, h, 2], np.float32)

    for i in range(0, w):
        for j in range(0, h):
            uv_d[i, j, 0] = j
            uv_d[i, j, 1] = i

    uv_d = np.expand_dims(uv_d.swapaxes(2, 1).swapaxes(1, 0), 0)     # 1 2 w h
    uv_d = torch.from_numpy(uv_d).cuda()
    uv_d = uv_d.repeat(b, 1, 1, 1)

    det_uv = uv_d + det_uv                                            # b 2 w h

    return det_uv

def uniform(shape, fish_uv):
    b, _, w, h = shape
    x0 = (w - 1) / 2. 

    fish_nor = (fish_uv - x0)/x0             # b 2 w h
    fish_nor = fish_nor.permute(0, 2, 3, 1)  # b w h 2
    return fish_nor

def resample_image(feature, flow):
    fish_uv = get_coordinate(feature.shape, flow)
    grid = uniform(feature.shape, fish_uv)
    target_image = F.grid_sample(feature, grid)
    return target_image

In [15]:
class Get_image(nn.Module):
    def __init__(self, input_dim, output_dim, activation='tanh'):
        super(Get_image, self).__init__()
        self.conv = Conv2dBlock(input_dim, output_dim, kernel_size=3, stride=1,
                     padding=1, pad_type='reflect', activation=activation)
    def forward(self, x):
        return self.conv(x)

class ResBlocks(nn.Module):
    def __init__(self, num_blocks, dim, norm='in', activation='relu', pad_type='zero', use_sn=False):
        super(ResBlocks, self).__init__()
        self.model = []
        for i in range(num_blocks):
            self.model += [ResBlock(dim, norm=norm, activation=activation, pad_type=pad_type, use_sn=use_sn)]
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x)

class ResBlock(nn.Module):
    def __init__(self, dim, norm='in', activation='relu', pad_type='zero', use_sn=False):
        super(ResBlock, self).__init__()

        model = []
        model += [Conv2dBlock(dim ,dim, 3, 1, 1, norm=norm, activation=activation, pad_type=pad_type, use_sn=use_sn)]
        model += [Conv2dBlock(dim ,dim, 3, 1, 1, norm=norm, activation='none', pad_type=pad_type, use_sn=use_sn)]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        residual = x
        out = self.model(x)
        out += residual
        return out

class DilationBlock(nn.Module):
    def __init__(self, dim, norm='in', activation='relu', pad_type='zero'):
        super(DilationBlock, self).__init__()

        model = []
        model += [Conv2dBlock(dim ,dim, 3, 1, 2, norm=norm, activation=activation, pad_type=pad_type, dilation=2)]
        model += [Conv2dBlock(dim ,dim, 3, 1, 4, norm=norm, activation=activation, pad_type=pad_type, dilation=4)]
        model += [Conv2dBlock(dim ,dim, 3, 1, 8, norm=norm, activation=activation, pad_type=pad_type, dilation=8)]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        out = self.model(x)
        return out

class Conv2dBlock(nn.Module):
    def __init__(self, input_dim ,output_dim, kernel_size, stride,
                 padding=0, norm='none', activation='relu', pad_type='zero', dilation=1,
                 use_bias=True, use_sn=False):
        super(Conv2dBlock, self).__init__()
        self.use_bias = use_bias
        # initialize padding
        if pad_type == 'reflect':
            self.pad = nn.ReflectionPad2d(padding)
        elif pad_type == 'replicate':
            self.pad = nn.ReplicationPad2d(padding)
        elif pad_type == 'zero':
            self.pad = nn.ZeroPad2d(padding)
        else:
            assert 0, "Unsupported padding type: {}".format(pad_type)

        # initialize normalization
        norm_dim = output_dim
        if norm == 'bn':
            self.norm = nn.BatchNorm2d(norm_dim)
        elif norm == 'in':
            self.norm = nn.InstanceNorm2d(norm_dim)
        elif norm == 'ln':
            self.norm = LayerNorm(norm_dim)
        elif norm == 'adain':
            self.norm = AdaptiveInstanceNorm2d(norm_dim)
        elif norm == 'none':
            self.norm = None
        else:
            assert 0, "Unsupported normalization: {}".format(norm)

        # initialize activation
        if activation == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU(0.2, inplace=True)
        elif activation == 'prelu':
            self.activation = nn.PReLU()
        elif activation == 'selu':
            self.activation = nn.SELU(inplace=True)
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'none':
            self.activation = None
        else:
            assert 0, "Unsupported activation: {}".format(activation)

        # initialize convolution
        if use_sn:
            self.conv = spectral_norm(nn.Conv2d(input_dim, output_dim, kernel_size, stride, bias=self.use_bias, dilation=dilation))
        else:
            self.conv = nn.Conv2d(input_dim, output_dim, kernel_size, stride, bias=self.use_bias, dilation=dilation)

    def forward(self, x):
        x = self.conv(self.pad(x))
        if self.norm:
            x = self.norm(x)
        if self.activation:
            x = self.activation(x)
        return x

class TransConv2dBlock(nn.Module):
    def __init__(self, input_dim ,output_dim, kernel_size, stride,
                 padding=0, norm='none', activation='relu'):
        super(TransConv2dBlock, self).__init__()
        self.use_bias = True

        # initialize normalization
        norm_dim = output_dim
        if norm == 'bn':
            self.norm = nn.BatchNorm2d(norm_dim)
        elif norm == 'in':
            self.norm = nn.InstanceNorm2d(norm_dim)
        elif norm == 'in_affine':
            self.norm = nn.InstanceNorm2d(norm_dim, affine=True)
        elif norm == 'ln':
            self.norm = LayerNorm(norm_dim)
        elif norm == 'adain':
            self.norm = AdaptiveInstanceNorm2d(norm_dim)
        elif norm == 'none':
            self.norm = None
        else:
            assert 0, "Unsupported normalization: {}".format(norm)

        # initialize activation
        if activation == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU(0.2, inplace=True)
        elif activation == 'prelu':
            self.activation = nn.PReLU()
        elif activation == 'selu':
            self.activation = nn.SELU(inplace=True)
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'none':
            self.activation = None
        else:
            assert 0, "Unsupported activation: {}".format(activation)

        # initialize convolution
        self.transConv = nn.ConvTranspose2d(input_dim, output_dim, kernel_size, stride, padding, bias=self.use_bias)

    def forward(self, x):
        x = self.transConv(x)
        if self.norm:
            x = self.norm(x)
        if self.activation:
            x = self.activation(x)
        return x

class AdaptiveInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(AdaptiveInstanceNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # weight and bias are dynamically assigned
        self.weight = None
        self.bias = None
        # just dummy buffers, not used
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        assert self.weight is not None and self.bias is not None, "Please assign weight and bias before calling AdaIN!"
        b, c = x.size(0), x.size(1)
        running_mean = self.running_mean.repeat(b)
        running_var = self.running_var.repeat(b)

        # Apply instance norm
        x_reshaped = x.contiguous().view(1, b * c, *x.size()[2:])

        out = F.batch_norm(
            x_reshaped, running_mean, running_var, self.weight, self.bias,
            True, self.momentum, self.eps)

        return out.view(b, c, *x.size()[2:])

    def __repr__(self):
        return self.__class__.__name__ + '(' + str(self.num_features) + ')'

class LayerNorm(nn.Module):
    def __init__(self, n_out, eps=1e-5, affine=True):
        super(LayerNorm, self).__init__()
        self.n_out = n_out
        self.affine = affine

        if self.affine:
          self.weight = nn.Parameter(torch.ones(n_out, 1, 1))
          self.bias = nn.Parameter(torch.zeros(n_out, 1, 1))

    def forward(self, x):
        normalized_shape = x.size()[1:]
        if self.affine:
          return F.layer_norm(x, normalized_shape, self.weight.expand(normalized_shape), self.bias.expand(normalized_shape))
        else:
          return F.layer_norm(x, normalized_shape)

class DownsampleResBlock(nn.Module):
    def __init__(self, input_dim, output_dim, norm='in', activation='relu', pad_type='zero', use_sn=False):
        super(DownsampleResBlock, self).__init__()
        self.conv_1 = nn.ModuleList()
        self.conv_2 = nn.ModuleList()

        self.conv_1.append(Conv2dBlock(input_dim,input_dim,3,1,1,'none',activation,pad_type,use_sn=use_sn))
        self.conv_1.append(Conv2dBlock(input_dim,output_dim,3,1,1,'none',activation,pad_type,use_sn=use_sn))
        self.conv_1.append(nn.AvgPool2d(kernel_size=2, stride=2))
        self.conv_1 = nn.Sequential(*self.conv_1)


        self.conv_2.append(nn.AvgPool2d(kernel_size=2, stride=2))
        self.conv_2.append(Conv2dBlock(input_dim,output_dim,1,1,0,'none',activation,pad_type,use_sn=use_sn))
        self.conv_2 = nn.Sequential(*self.conv_2)


    def forward(self, x):
        out = self.conv_1(x) + self.conv_2(x)
        return out

In [16]:
class BaseNetwork(nn.Module):
  def __init__(self):
    super(BaseNetwork, self).__init__()

  def print_network(self):
    if isinstance(self, list):
      self = self[0]
    num_params = 0
    for param in self.parameters():
      num_params += param.numel()
    print('Network [%s] was created. Total number of parameters: %.1f million. '
          'To see the architecture, do print(network).'% (type(self).__name__, num_params / 1000000))

  def init_weights(self, init_type='normal', gain=0.02):
    '''
    initialize network's weights
    init_type: normal | xavier | kaiming | orthogonal
    https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/9451e70673400885567d08a9e97ade2524c700d0/models/networks.py#L39
    '''
    def init_func(m):
      classname = m.__class__.__name__
      if classname.find('InstanceNorm2d') != -1:
        if hasattr(m, 'weight') and m.weight is not None:
          nn.init.constant_(m.weight.data, 1.0)
        if hasattr(m, 'bias') and m.bias is not None:
          nn.init.constant_(m.bias.data, 0.0)
      elif hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
        if init_type == 'normal':
          nn.init.normal_(m.weight.data, 0.0, gain)
        elif init_type == 'xavier':
          nn.init.xavier_normal_(m.weight.data, gain=gain)
        elif init_type == 'xavier_uniform':
          nn.init.xavier_uniform_(m.weight.data, gain=1.0)
        elif init_type == 'kaiming':
          nn.init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
        elif init_type == 'orthogonal':
          nn.init.orthogonal_(m.weight.data, gain=gain)
        elif init_type == 'none':  # uses pytorch's default init method
          m.reset_parameters()
        else:
          raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
        if hasattr(m, 'bias') and m.bias is not None:
          nn.init.constant_(m.bias.data, 0.0)

    self.apply(init_func)

    # propagate to children
    for m in self.children():
      if hasattr(m, 'init_weights'):
        m.init_weights(init_type, gain)

class InpaintGenerator(nn.Module):
  def __init__(self):
    super(InpaintGenerator, self).__init__()

    self.flow_column = FlowColumn()
    self.conv_column = InpaintNet()

  def forward(self, inputs):
      flow_map, flows = self.flow_column(inputs)
      pyramid_imgs, images_out = self.conv_column(inputs, flows)
      return pyramid_imgs, images_out,flow_map

class InpaintNet(BaseNetwork):
  def __init__(self, init_weights=True):
    super(InpaintNet, self).__init__()

    cnum = 32

    self.dw_conv01 = nn.Sequential(
        nn.Conv2d(3, cnum, kernel_size=3, stride=2, padding=1),
        nn.LeakyReLU(0.2, inplace=True))
    self.dw_conv02 = nn.Sequential(
        nn.Conv2d(cnum, cnum * 2, kernel_size=3, stride=2, padding=1),
        nn.LeakyReLU(0.2, inplace=True))
    self.dw_conv03 = nn.Sequential(
        nn.Conv2d(cnum * 2, cnum * 4, kernel_size=3, stride=2, padding=1),
        nn.LeakyReLU(0.2, inplace=True))
    self.dw_conv04 = nn.Sequential(
        nn.Conv2d(cnum * 4, cnum * 8, kernel_size=3, stride=2, padding=1),
        nn.LeakyReLU(0.2, inplace=True))
    self.dw_conv05 = nn.Sequential(
        nn.Conv2d(cnum * 8, cnum * 16, kernel_size=3, stride=2, padding=1),
        nn.LeakyReLU(0.2, inplace=True))
    self.dw_conv06 = nn.Sequential(
        nn.Conv2d(cnum * 16, cnum * 16, kernel_size=3, stride=2, padding=1),
        nn.ReLU(inplace=True))

    self.up_conv05 = nn.Sequential(
        nn.Conv2d(cnum * 16, cnum * 16, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True))
    self.up_conv04 = nn.Sequential(
        nn.Conv2d(cnum * 32, cnum * 8, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True))
    self.up_conv03 = nn.Sequential(
        nn.Conv2d(cnum * 16, cnum * 4, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True))
    self.up_conv02 = nn.Sequential(
        nn.Conv2d(cnum * 8, cnum * 2, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True))
    self.up_conv01 = nn.Sequential(
        nn.Conv2d(cnum * 4, cnum * 1, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True))

    self.decoder = nn.Sequential(
        nn.Conv2d(cnum * 2, cnum, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(cnum, 3, kernel_size=3, stride=1, padding=1),
        nn.Tanh())

    self.torgb5 = nn.Sequential(
        nn.Conv2d(cnum * 32, 3, kernel_size=1, stride=1, padding=0),
        nn.Tanh())
    self.torgb4 = nn.Sequential(
        nn.Conv2d(cnum * 16, 3, kernel_size=1, stride=1, padding=0),
        nn.Tanh())
    self.torgb3 = nn.Sequential(
        nn.Conv2d(cnum * 8, 3, kernel_size=1, stride=1, padding=0),
        nn.Tanh())
    self.torgb2 = nn.Sequential(
        nn.Conv2d(cnum * 4, 3, kernel_size=1, stride=1, padding=0),
        nn.Tanh())
    self.torgb1 = nn.Sequential(
        nn.Conv2d(cnum * 2, 3, kernel_size=1, stride=1, padding=0),
        nn.Tanh())

    if init_weights:
        self.init_weights()

  def forward(self, img, flows):
      x = img

      x1 = self.dw_conv01(x)
      x2 = self.dw_conv02(x1)
      x3 = self.dw_conv03(x2)
      x4 = self.dw_conv04(x3)
      x5 = self.dw_conv05(x4)
      x6 = self.dw_conv06(x5)

      x5 = resample_image(x5, flows[4])
      x4 = resample_image(x4, flows[3])
      x3 = resample_image(x3, flows[2])
      x2 = resample_image(x2, flows[1])
      x1 = resample_image(x1, flows[0])

      upx5 = self.up_conv05(F.interpolate(x6, scale_factor=2, mode='bilinear', align_corners=True))
      upx4 = self.up_conv04(
          F.interpolate(torch.cat([upx5, x5], dim=1), scale_factor=2, mode='bilinear', align_corners=True))
      upx3 = self.up_conv03(
          F.interpolate(torch.cat([upx4, x4], dim=1), scale_factor=2, mode='bilinear', align_corners=True))
      upx2 = self.up_conv02(
          F.interpolate(torch.cat([upx3, x3], dim=1), scale_factor=2, mode='bilinear', align_corners=True))
      upx1 = self.up_conv01(
          F.interpolate(torch.cat([upx2, x2], dim=1), scale_factor=2, mode='bilinear', align_corners=True))

      img5 = self.torgb5(torch.cat([upx5, x5], dim=1))
      img4 = self.torgb4(torch.cat([upx4, x4], dim=1))
      img3 = self.torgb3(torch.cat([upx3, x3], dim=1))
      img2 = self.torgb2(torch.cat([upx2, x2], dim=1))
      img1 = self.torgb1(torch.cat([upx1, x1], dim=1))

      output = self.decoder(
          F.interpolate(torch.cat([upx1, x1], dim=1), scale_factor=2, mode='bilinear', align_corners=True))
      pyramid_imgs = [img1, img2, img3, img4, img5]
      return pyramid_imgs, output

class FlowColumn(nn.Module):
    def __init__(self, input_dim=3, dim=64, n_res=2, activ='lrelu',
                 norm='in', pad_type='reflect', use_sn=True):
        super(FlowColumn, self).__init__()

        self.down_flow01 = nn.Sequential(
            Conv2dBlock(input_dim, dim // 2, 7, 1, 3, norm, activ, pad_type, use_sn=use_sn),
            Conv2dBlock(dim // 2, dim // 2, 4, 2, 1, norm, activ, pad_type, use_sn=use_sn))
        self.down_flow02 = nn.Sequential(
            Conv2dBlock(dim // 2, dim // 2, 4, 2, 1, norm, activ, pad_type, use_sn=use_sn))
        self.down_flow03 = nn.Sequential(
            Conv2dBlock(dim // 2, dim, 4, 2, 1, norm, activ, pad_type, use_sn=use_sn))
        self.down_flow04 = nn.Sequential(
            Conv2dBlock(dim, 2 * dim, 4, 2, 1, norm, activ, pad_type, use_sn=use_sn))
        self.down_flow05 = nn.Sequential(
            Conv2dBlock(2 * dim, 4 * dim, 4, 2, 1, norm, activ, pad_type, use_sn=use_sn))
        self.down_flow06 = nn.Sequential(
            Conv2dBlock(4 * dim, 8 * dim, 4, 2, 1, norm, activ, pad_type, use_sn=use_sn))

        dim = 8 * dim

        self.up_flow05 = nn.Sequential(
            ResBlocks(n_res, dim, norm, activ, pad_type=pad_type),
            TransConv2dBlock(dim, dim // 2, 6, 2, 2, norm=norm, activation=activ))

        self.up_flow04 = nn.Sequential(
            Conv2dBlock(dim, dim // 2, 5, 1, 2, norm, activ, pad_type, use_sn=use_sn),
            ResBlocks(n_res, dim // 2, norm, activ, pad_type=pad_type),
            TransConv2dBlock(dim // 2, dim // 4, 6, 2, 2, norm=norm, activation=activ))

        self.up_flow03 = nn.Sequential(
            Conv2dBlock(dim // 2, dim // 4, 5, 1, 2, norm, activ, pad_type, use_sn=use_sn),
            ResBlocks(n_res, dim // 4, norm, activ, pad_type=pad_type),
            TransConv2dBlock(dim // 4, dim // 8, 6, 2, 2, norm=norm, activation=activ))

        self.up_flow02 = nn.Sequential(
            Conv2dBlock(dim // 4, dim // 8, 5, 1, 2, norm, activ, pad_type, use_sn=use_sn),
            ResBlocks(n_res, dim // 8, norm, activ, pad_type=pad_type),
            TransConv2dBlock(dim // 8, dim // 16, 6, 2, 2, norm=norm, activation=activ))

        self.up_flow01 = nn.Sequential(
            Conv2dBlock(dim // 8, dim // 16, 5, 1, 2, norm, activ, pad_type, use_sn=use_sn),
            ResBlocks(n_res, dim // 16, norm, activ, pad_type=pad_type),
            TransConv2dBlock(dim // 16, dim // 16, 6, 2, 2, norm=norm, activation=activ))

        self.location = nn.Sequential(
            Conv2dBlock(dim // 8, dim // 16, 5, 1, 2, norm, activ, pad_type, use_sn=use_sn),
            ResBlocks(n_res, dim // 16, norm, activ, pad_type=pad_type),
            TransConv2dBlock(dim // 16, dim // 16, 6, 2, 2, norm=norm, activation=activ),
            Conv2dBlock(dim // 16, 2, 3, 1, 1, norm='none', activation='none', pad_type=pad_type, use_bias=False))

        self.to_flow05 = Conv2dBlock(dim // 2, 2, 3, 1, 1, norm='none', activation='none', pad_type=pad_type, use_bias=False)
        self.to_flow04 = Conv2dBlock(dim // 4, 2, 3, 1, 1, norm='none', activation='none', pad_type=pad_type, use_bias=False)
        self.to_flow03 = Conv2dBlock(dim // 8, 2, 3, 1, 1, norm='none', activation='none', pad_type=pad_type, use_bias=False)
        self.to_flow02 = Conv2dBlock(dim // 16, 2, 3, 1, 1, norm='none', activation='none', pad_type=pad_type, use_bias=False)
        self.to_flow01 = Conv2dBlock(dim // 16, 2, 3, 1, 1, norm='none', activation='none', pad_type=pad_type, use_bias=False)

    def forward(self, inputs):
        f_x1 = self.down_flow01(inputs)
        f_x2 = self.down_flow02(f_x1)
        f_x3 = self.down_flow03(f_x2)
        f_x4 = self.down_flow04(f_x3)
        f_x5 = self.down_flow05(f_x4)
        f_x6 = self.down_flow06(f_x5)

        f_u5 = self.up_flow05(f_x6)
        f_u4 = self.up_flow04(torch.cat((f_u5, f_x5), 1))
        f_u3 = self.up_flow03(torch.cat((f_u4, f_x4), 1))
        f_u2 = self.up_flow02(torch.cat((f_u3, f_x3), 1))
        f_u1 = self.up_flow01(torch.cat((f_u2, f_x2), 1))
        flow_map = self.location(torch.cat((f_u1, f_x1), 1))

        flow05 = self.to_flow05(f_u5)
        flow04 = self.to_flow04(f_u4)
        flow03 = self.to_flow03(f_u3)
        flow02 = self.to_flow02(f_u2)
        flow01 = self.to_flow01(f_u1)

        return flow_map, [flow01, flow02, flow03, flow04, flow05]


In [90]:
# flownet train

def main():
    transform = A.Compose(
    [   
        A.OneOf([RandomFisheye()],p=1),
        A.Resize(train_size[1], train_size[0]),
        A.Normalize(),
        ToTensorV2()
    ])
    # Data loading code
    train_dataset = CustomDataset(csv_file='./train_source.csv', transform=transform, source=True)

    train_source_loader = DataLoader(train_dataset, batch_size=f_batch_size,
                                     shuffle=True, num_workers=1, pin_memory=True, drop_last=True)

    flownet = InpaintGenerator().to(device)
    l1_loss = nn.L1Loss()
    optimizer = torch.optim.Adam(flownet.parameters(), lr=1e-4,betas=(0.5, 0.999))
    lr_scheduler = LambdaLR(optimizer, lambda F_epoch : 0.95**F_epoch )
    best_mae = 9999
    
    #print(summary(flownet,torch.zeros(1,3,512,512).to(device),max_depth=None,show_parent_layers=True,show_input=True))

    for F_epoch in range(F_epochs):
        print("current epoch : ", F_epoch)
        start_time = time.time()
        epoch_loss = 0
        mae = 0
        for images, gt in tqdm(train_source_loader):

            images, gt = set_device([images, gt])
            gt = gt.permute(0,3,1,2).type(torch.DoubleTensor).to(device)
            feats, pred_img,flow_map = flownet(images)
            comp_img = pred_img

            #reconstruction loss
            L1_loss = l1_loss(comp_img, gt) * l1_weight

            if feats is not None:
                pyramid_loss = 0
                for _, f in enumerate(feats):
                    #print(f)
                    pyramid_loss += l1_loss(f, F.interpolate(gt, size=f.size()[2:4], mode='bilinear',
                                                                  align_corners=True))
                pyramid_loss += pyramid_loss * pyramid_weight

            flow_loss = L1_loss+pyramid_loss
        
            # generator backward
            optimizer.zero_grad()
            flow_loss.backward()
            optimizer.step()

            epoch_loss += flow_loss.item()

        print(f'Epoch {F_epoch}, Loss: {epoch_loss/len(train_source_loader)}')

        lr_scheduler.step()

        new_mae = (torch.mean(torch.abs(gt - pred_img))).item()
        mae = new_mae
        print("flow_loss : ", flow_loss.item())
        if mae < best_mae :
            torch.save({
            'model': flownet.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'epoch': F_epoch,
            }, f'./flow_save_path/flow_save_{F_epoch}.pth' )
            print("save_model / mae : ", mae)
            best_mae = mae

        end_time = time.time()
        print("epoch_time : ", end_time-start_time)

In [40]:
#main()

## ADVENT train

In [17]:
def main(resume=None):
    cudnn.benchmark = True

    transform = A.Compose(
    [   
        Fisheye(),
        A.Resize(train_size[1], train_size[0]),
        A.augmentations.geometric.transforms.HorizontalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2()
    ])
    # Data loading code
    source_dataset = CustomDataset(csv_file='./train_source.csv', transform=transform, source=True)

    train_source_loader = DataLoader(source_dataset, batch_size=batch_size,
                                     shuffle=True, num_workers=1, pin_memory=True, drop_last=True)

    target_dataset = CustomDataset(csv_file='./train_target.csv', transform=transform, source=False)
    train_target_loader = DataLoader(target_dataset, batch_size=batch_size,
                                     shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
    
    val_target_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform)
    val_target_loader = DataLoader(val_target_dataset, batch_size=1, shuffle=False, pin_memory=True)

    train_source_iter = ForeverDataIterator(train_source_loader)
    train_target_iter = ForeverDataIterator(train_target_loader)

    # create model
    num_classes = num_class
    model = deeplabv2_resnet101(num_classes=num_classes).to(device)
    discriminator = Discriminator(num_classes=num_classes).to(device)
    flownet = InpaintGenerator().to(device)
    flow_path = './flow_save_path/flow_save_27.pth'
    flow_ckpt = torch.load(flow_path, map_location='cpu')
    flownet.load_state_dict(flow_ckpt['model'])
    flownet.eval()
    
    # define optimizer and lr scheduler
    optimizer = SGD(model.get_parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    optimizer_d = Adam(discriminator.parameters(), lr=lr_d, betas=(0.9, 0.99))
    lr_scheduler = LambdaLR(optimizer, lambda x: learning_rate * (1. - float(x) / epochs / iters_per_epoch) ** (lr_power))
    lr_scheduler_d = LambdaLR(optimizer_d, lambda x: (1. - float(x) / epochs / iters_per_epoch) ** (lr_power))

    #optionally resume from a checkpoint
    if resume:
        checkpoint = torch.load(resume, map_location='cpu')
        model.load_state_dict(checkpoint['model'])
        discriminator.load_state_dict(checkpoint['discriminator'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        optimizer_d.load_state_dict(checkpoint['optimizer_d'])
        lr_scheduler_d.load_state_dict(checkpoint['lr_scheduler_d'])
        start_epoch = checkpoint['epoch'] + 1
    else:
        start_epoch =0
    # define loss function (criterion)
    criterion = torch.nn.CrossEntropyLoss(ignore_index=ignore_label).to(device) # segmentation loss
    dann = DomainAdversarialEntropyLoss(discriminator) # adversarial loss 
    interp_train = nn.Upsample(size=train_size[::-1], mode='bilinear', align_corners=True)
    interp_val = nn.Upsample(size=test_output_size[::-1], mode='bilinear', align_corners=True)


    # define visualization function
    #decode = source_dataset.decode_target

    # def visualize(image, pred, label, prefix):
    #     """
    #     Args:
    #         image (tensor): 3 x H x W
    #         pred (tensor): C x H x W
    #         label (tensor): H x W
    #         prefix: prefix of the saving image
    #     """
    #     image = image.detach().cpu().numpy()
    #     pred = pred.detach().max(dim=0)[1].cpu().numpy()
    #     label = label.cpu().numpy()
    #     for tensor, name in [TypeError: 'int' object is not callable
    #         (Image.fromarray(np.uint8(DeNormalizeAndTranspose()(image))), "image"),
    #         (decode(label), "label"),
    #         (decode(pred), "pred")
    #     ]:
    #         tensor.save(logger.get_image_path("{}_{}.png".format(prefix, name)))

    # if args.phase == 'test':
    #     confmat = validate(val_target_loader, model, interp_val, criterion, visualize, args)
    #     print(confmat)
    #     return

    # start training
    best_iou = 0.
    for epoch in range(start_epoch, epochs):
        print("current epoch :", epoch)
        print(lr_scheduler.get_lr(), lr_scheduler_d.get_lr())
        # train for one epoch
        print("train start")
        train_start = time.time()
        train(train_source_iter, train_target_iter, model, interp_train, criterion, dann,flownet, optimizer,
              lr_scheduler, optimizer_d, lr_scheduler_d, epoch)
        train_end = time.time()
        print("train end")
        print("epoch_time : ", train_end-train_start)
        # evaluate on validation set
        confmat = validate(val_target_loader, model, interp_train, criterion)
        print(confmat.format(source_dataset.classes))
        acc_global, acc, iu = confmat.compute()

        # calculate the mean iou over partial classes
        indexes = [source_dataset.classes.index(name) for name
                   in source_dataset.classes]
        iu = iu[indexes]
        mean_iou = iu.mean()

        #remember best acc@1 and save checkpoint
        if mean_iou > best_iou:
            torch.save(
            {
                'model': model.state_dict(),
                'discriminator': discriminator.state_dict(),
                'optimizer': optimizer.state_dict(),
                'optimizer_d': optimizer_d.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'lr_scheduler_d': lr_scheduler_d.state_dict(),
                'epoch': epoch,
            }, f'./save_path/model_save_v2_flow_{epoch}.pth' )
        
    
            #shutil.copy(logger.get_checkpoint_path(epoch), logger.get_checkpoint_path('best'))
        best_iou = max(best_iou, mean_iou)
        print("Target: {} Best: {}".format(mean_iou, best_iou))


## Run

In [18]:
re_checkpoint = './save_path/model_save_v2_flow_19.pth'
main(re_checkpoint)
main()

/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torch/optim/sgd.py:27: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super().__init__(params, defaults)


current epoch : 20
[0.0015786146687233882, 0.015786146687233882] [6.314458674893553e-05]
train start
iters :  0


/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/home/jisu/anaconda3/envs/dinov2/lib/python3.9/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


seg_loss :  0.13430733978748322
entropy_loss:  1.5470466613769531
disc_loss :  0.4074442982673645
iter end, time :  7.900942802429199
iters :  100
seg_loss :  0.10852577537298203
entropy_loss:  1.5860178470611572
disc_loss :  0.546929121017456
iter end, time :  1.0153617858886719
iters :  200
seg_loss :  0.13114376366138458
entropy_loss:  1.3395910263061523
disc_loss :  0.43961653113365173
iter end, time :  1.0197322368621826
iters :  300
seg_loss :  0.07959704101085663
entropy_loss:  1.087096929550171
disc_loss :  0.5469233989715576
iter end, time :  1.0167593955993652
iters :  400
seg_loss :  0.11285069584846497
entropy_loss:  1.2388653755187988
disc_loss :  0.5443356037139893
iter end, time :  0.9651339054107666
iters :  500
seg_loss :  0.1507871448993683
entropy_loss:  1.4134207963943481
disc_loss :  0.6279618740081787
iter end, time :  1.0077173709869385
iters :  600
seg_loss :  0.16402029991149902
entropy_loss:  1.1789054870605469
disc_loss :  0.5853868722915649
iter end, time : 

KeyboardInterrupt: 

##                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Entropy Map Visualization

In [ ]:
# target_layers = [model.model.blocks[-1].norm1]

# def reshape_transform(tensor, height=16, width=16):
#     result = tensor[:, 1 :  , :].reshape(tensor.size(0),
#         height, width, tensor.size(2))

#     # Bring the channels to the first dimension,
#     # like in CNNs.
#     result = result.transpose(2, 3).transpose(1, 2)
#     return result

# cam = GradCAM(model=model2.model, target_layers=target_layers, reshape_transform=reshape_transform)

# def analysis_image(image, target_class):
#   img = cv2.resize(image, [224, 224])
#   img = np.expand_dims(img, axis = 0)
#   img = torch.FloatTensor(img).permute(0,3,1,2)
#   input_tensor = img.cuda()

#   targets = [ClassifierOutputTarget(target_class)]

#   Cam = cam(input_tensor = input_tensor, targets = targets)

#   img2 = cv2.resize(image, [224, 224]) / 255.

#   Cam = np.expand_dims(np.squeeze(Cam, axis = 0), axis = -1)

#   visualization = show_cam_on_image(img2.astype(np.float32), Cam, use_rgb = True)

#   # colab에서 실행하기 때문에 cv2.imshow 대신 cv2_imshow 함수 이용
#   cv2_imshow(visualization)

## Inference

In [19]:
mode = 'test'
transform = A.Compose(
    [   
        A.Resize(train_size[0], train_size[1]),
        A.Normalize(),
        ToTensorV2()
    ])
if mode == 'val':
    test_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform, infer=True)
elif mode == 'test':
    test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)#
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=1)

In [21]:
def inference(save_model):
    interp_val = nn.Upsample(size=test_output_size[::-1], mode='bilinear', align_corners=True)
    with torch.no_grad():
        num_classes = num_class
        model = deeplabv2_resnet101(num_classes=num_classes).to(device)
        model.load_state_dict(save_model)
        model.eval()
        result = []
        iter=0
        batch_size=16
        for images in tqdm(test_dataloader):
            images = images.float().to(device)
            outputs = model(images)
            outputs = interp_val(outputs)
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()
            # batch에 존재하는 각 이미지에 대해서 반복
            for i,pred in enumerate(outputs,1):
                mask_img = np.zeros((540,960,3),dtype=np.uint8)
                pred = pred.astype(np.uint8)
                pred = Image.fromarray(pred) # 이미지로 변환
                pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
                pred = np.array(pred) # 다시 수치로 변환
                # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
                for class_id in range(12):
                    class_mask = (pred == class_id).astype(np.uint8)
                    mask_img = palette[pred]
                    # if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    #     mask_rle = rle_encode(class_mask)
                    #     result.append(mask_rle)
                    # else: # 마스크가 존재하지 않는 경우 -1
                    #     result.append(-1)

                if mode == 'val':
                    test = pd.read_csv('./val_source.csv')
                elif mode == 'test':
                    test = pd.read_csv('./test.csv')
                img_name=test['id'][iter*16+i-1]
                img_org = cv2.imread(test['img_path'][iter*16+i-1])
                img_org = cv2.resize(img_org,(960,540))
            
                result = np.hstack((img_org,mask_img))
                if mode == 'val':
                    new_gt = np.zeros((540,960,3),dtype=np.uint8)
                    img_gt = cv2.imread(test['gt_path'][iter*16+i-1],0)
                    img_gt = cv2.resize(img_gt,(960,540))
                    mask_255 = np.where(img_gt>=12)
                    img_gt[mask_255] = 12
                    new_gt = palette[img_gt]
                    result = np.hstack((img_org,mask_img,new_gt))
                    if not os.path.exists('./mask_save_s'):
                        os.makedirs('./mask_save_s')
                    cv2.imwrite(f"./mask_save_s/{img_name}.png",result)
                elif mode == 'test':    
                    if not os.path.exists('./mask_save'):
                        os.makedirs('./mask_save')
                    cv2.imwrite(f"./mask_save/{img_name}.png",result)

            iter+=1

    return result


In [22]:
model_path = './save_path/model_save_v2_flow_27.pth'
checkpoint = torch.load(model_path, map_location='cpu')
result = inference(checkpoint['model'])

  1%|          | 1/119 [00:11<22:48, 11.60s/it]


KeyboardInterrupt: 

## Submission

In [24]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit

,id,mask_rle
0,TEST_0000_class_0,453748 5 454705 10 454739 5 455663 14 455696 1...
1,TEST_0000_class_1,442379 2 443278 6 443337 5 444235 13 444267 8 ...
2,TEST_0000_class_2,1 6 18 100 602 355 961 13 977 101 1561 358 192...
3,TEST_0000_class_3,496650 12 496674 8 497624 4 498583 6 499543 6 ...
4,TEST_0000_class_4,-1
...,...,...
22771,TEST_1897_class_7,125776 1 126736 2 127695 3 128655 4 129615 4 1...
22772,TEST_1897_class_8,94 543 678 126 1053 543 1639 124 2012 543 2600...
22773,TEST_1897_class_9,-1
22774,TEST_1897_class_10,-1


In [25]:
submit.to_csv('./flow19_submit.csv', index=False)